<a href="https://colab.research.google.com/github/yilunz/deep-learning-uiuc/blob/master/hw4_resnet_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import torch.distributed as dist

import os
import subprocess
from mpi4py import MPI
import h5py
import time

In [54]:
batch_size=128
LR=0.001
Num_Epochs=100
num_output=100
scheduler_step_size=20
scheduler_gamma=0.1
c=3
Kernel_Size=3

transform_train=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.ToTensor()])
transform_test=transforms.ToTensor()

#Load CIFAR 100 Dataset
trainset=torchvision.datasets.CIFAR100(root='~/scratch/',train=True, download=True,transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=8)

testset=torchvision.datasets.CIFAR100(root='~/scratch/',train=False,download=True,transform=transform_test)
testloader=torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False,num_workers=8)



Files already downloaded and verified
Files already downloaded and verified


In [0]:
class BasicBlock(nn.Module):
  def __init__(self,inplanes,planes,stride,padding,downsample=None):
    super(BasicBlock,self).__init__()
    self.conv1=nn.Conv2d(inplanes,planes,kernel_size=3,stride=stride,padding=padding)
    #self.conv1=conv3x3(inplanes,planes,stride)
    self.bn1=nn.BatchNorm2d(planes)
    self.relu=nn.ReLU(inplace=True)
    self.conv2=nn.Conv2d(planes,planes,kernel_size=3,stride=1,padding=padding) #stride should be 1 for the second conv
    #self.conv2=conv3x3(planes,planes)
    self.bn2=nn.BatchNorm2d(planes)
    self.downsample=downsample
    self.stride=stride
  def forward(self,x):
    residual=x
    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
    out=self.conv2(out)
    out=self.bn2(out)
    if self.downsample is not None:
      residual = self.downsample(x)
    #print(residual.shape)
    #print(out.shape)
    out+=residual
    out=self.relu(out)
    return out


In [0]:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet,self).__init__()
    self.conv3=nn.Conv2d(3,32,3,stride=1,padding=1)#32, kernel_size=3, stride=1, padding=1 [32,32,32]
    self.bn3=nn.BatchNorm2d(32)
    self.relu1=nn.ReLU()
    self.dropout=nn.Dropout(p=0.5)

    #self.downsample1=nn.Conv2d(3,32,kernel_size=1,stride=1)
    self.bb1=BasicBlock(32,32,1,1,downsample=None) 
    self.bb2=BasicBlock(32,32,1,1,downsample=None)
    
    self.downsample1=nn.Conv2d(32,64,kernel_size=1,stride=2)
    self.bb3=BasicBlock(32,64,2,1,downsample=self.downsample1)
    self.bb4=BasicBlock(64,64,1,1,downsample=None)
    self.bb5=BasicBlock(64,64,1,1,downsample=None)
    self.bb6=BasicBlock(64,64,1,1,downsample=None)

    self.downsample2=nn.Conv2d(64,128,kernel_size=1,stride=2)
    self.bb7=BasicBlock(64,128,2,1,downsample=self.downsample2)
    self.bb8=BasicBlock(128,128,1,1,downsample=None)
    self.bb9=BasicBlock(128,128,1,1,downsample=None)
    self.bb10=BasicBlock(128,128,1,1,downsample=None)

    self.downsample3=nn.Conv2d(128,256,kernel_size=1,stride=2)
    self.bb11=BasicBlock(128,256,2,1,downsample=self.downsample3)
    self.bb12=BasicBlock(256,256,1,1,downsample=None)

    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=1)#2*2*256
    self.fc1=nn.Linear(2*2*256,num_output)


  def forward(self,x):
    out=self.conv3(x) #output size=32,32,32
    out=self.bn3(out)
    out=self.relu1(out)
    out=self.dropout(out)
    #print(out.size)

    #stack1
    out=self.bb1(out)#(in_channels,out_channels,stride)#[32,32,1]
    out=self.bb2(out)

    #stack2
    out=self.bb3(out)
    out=self.bb4(out)
    out=self.bb5(out)
    out=self.bb6(out)

    #stack3
    out=self.bb7(out)
    out=self.bb8(out)
    out=self.bb9(out)
    out=self.bb10(out)

    #stack4
    out=self.bb11(out)
    out=self.bb12(out)

    #max pool
    out=self.maxpool(out)
    #print(out.shape)

    #fully connected
    out = out.view(-1,2*2*256) #flatten
    out=self.fc1(out)
    #print(out.shape)
    return out



In [0]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#basic_block=BasicBlock().to(device)
resnet=ResNet().to(device)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.RMSprop(resnet.parameters(),lr=LR,weight_decay=0.0005)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=scheduler_step_size,gamma=scheduler_gamma)

start_time=time.time()

for epoch in range(Num_Epochs):
  scheduler.step()
  resnet.train()
  total = 0
  correct = 0
  start_time = time.time()
  for images,labels in trainloader:
    images=images.to(device)
    labels=labels.to(device)
    outputs=resnet(images)
    #print(outputs.shape)
    
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    _,predicted = torch.max(outputs,1)
    total += labels.size(0)
    
    correct += (predicted == labels).sum().item()
  
    loss.backward()
    optimizer.step()
  
  train_accuracy = correct/total
  
  with torch.no_grad():
    resnet.eval()
    correct=0
    total=0
    for images,labels in testloader:
      #images,labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = resnet(images)

      total += labels.size(0)
      _,predicted = torch.max(outputs,1)
      correct += (predicted == labels).sum().item()
    test_accuracy = correct/total
  
  print("Epoch {0}, Time {1:.4f}, Train Acc {2:.4f}, Test Acc {3:.4f}".format(epoch, round(time.time()-start_time,4), round(train_accuracy,4), round(test_accuracy,4)))
  torch.save(resnet.state_dict(),'epoch-{}.ckpt'.format(epoch))




/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 0, Time 78.7817, Train Acc 0.0390, Test Acc 0.0759
Epoch 1, Time 78.5673, Train Acc 0.1109, Test Acc 0.1129
Epoch 2, Time 78.5048, Train Acc 0.1677, Test Acc 0.1853
Epoch 3, Time 78.6230, Train Acc 0.2125, Test Acc 0.1935
Epoch 4, Time 78.3645, Train Acc 0.2509, Test Acc 0.1916
Epoch 5, Time 78.2334, Train Acc 0.2888, Test Acc 0.2025
Epoch 6, Time 78.0875, Train Acc 0.3222, Test Acc 0.2739
Epoch 7, Time 78.0031, Train Acc 0.3535, Test Acc 0.2999
Epoch 8, Time 77.7759, Train Acc 0.3776, Test Acc 0.3303
Epoch 9, Time 77.8351, Train Acc 0.4013, Test Acc 0.3427
Epoch 10, Time 77.8939, Train Acc 0.4226, Test Acc 0.3441
Epoch 11, Time 77.6501, Train Acc 0.4390, Test Acc 0.3478
Epoch 12, Time 77.7096, Train Acc 0.4562, Test Acc 0.3426
Epoch 13, Time 77.9748, Train Acc 0.4678, Test Acc 0.3329
Epoch 14, Time 77.5437, Train Acc 0.4831, Test Acc 0.3343
Epoch 15, Time 77.5739, Train Acc 0.4925, Test Acc 0.3901
Epoch 16, Time 77.5559, Train Acc 0.5091, Test Acc 0.3859
Epoch 17, Time 77.7172, 